In [14]:
import numpy as np
import pandas as pd
import os
import random

def set_seed(seed: int):
    random.seed(seed) # Python
    np.random.seed(seed)  # Numpy, é o gerador utilizado pelo sklearn
    os.environ["PYTHONHASHSEED"] = str(seed)  # sistema operativo

set_seed(25)

# Making our own embedding

In [18]:
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Flatten, Dense, Embedding, Input

# Parameters
max_features = 10000  # Tamanho do vocabulario
maxlen = 500  # sequencia maxima, posso testar com mais se o computador aguentar

# Load dos dados
csv_path = '../human_or_ai_dataset_small.csv'  # Change this to your file path
df = pd.read_csv(csv_path)

# Sanity check!
print("Dataset shape:", df.shape)
print("Columns:", df.columns)

# Separar os textos das labels
texts = df['text'].values
labels = df['source'].values

# Criar um tokenizer
tokenizer = preprocessing.text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(texts)

# Converter os textos para sequencias de inteiros
sequences = tokenizer.texts_to_sequences(texts)

# Padding para uniformizar tamanhos
x_data = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen)

# Garantir label numerica
if not np.issubdtype(labels.dtype, np.number):
    label_map = {label: i for i, label in enumerate(np.unique(labels))}
    y_data = np.array([label_map[label] for label in labels])
else:
    y_data = labels

# Data split !
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

# Check shapes
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

# Print a sample
print("Sample sequence:", x_train[0])
print("Sample label:", y_train[0])

"""
# If you want to save the tokenizer for later use
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
"""

Dataset shape: (5051, 2)
Columns: Index(['text', 'source'], dtype='object')
x_train shape: (4040, 500)
y_train shape: (4040,)
x_test shape: (1011, 500)
y_test shape: (1011,)
Sample sequence: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0   

"\n# If you want to save the tokenizer for later use\nimport pickle\nwith open('tokenizer.pickle', 'wb') as handle:\n    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)\n"

## Define and train the model

In [19]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, Dropout, GlobalMaxPooling1D, Conv1D, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

embedding_dims = 100  # Size of embedding vector

# Model with more sophisticated architecture
model = Sequential()

# Embedding layer
model.add(Input((maxlen,)))
model.add(Embedding(max_features, embedding_dims))
model.add(SpatialDropout1D(0.2))  # Spatial dropout evita overfitting nos embeddings


model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile with better optimizer
model.compile(
    optimizer='adam',  # Adam typically works better than rmsprop
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


# Early stopping 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Save best model
model_checkpoint = ModelCheckpoint(
    'best_model_embedding.h5',
    monitor='val_accuracy',
    save_best_only=True
)

# Train with callbacks
history = model.fit(
    x_train, y_train,
    epochs=15,  # More epochs, early stopping will prevent overfitting
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 500, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_4             │ (None, 500, 100)       │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 496, 128)       │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,072,449 (4.09 MB)

 Trainable params: 1,072,449 (4.09 MB)

 Non-trainable params: 0 (0.00 B)

...

101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 0.6700 - loss: 0.5769 - val_accuracy: 0.9072 - val_loss: 0.2539
Epoch 2/15
100/101 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.9368 - loss: 0.1810

101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 0.9368 - loss: 0.1808 - val_accuracy: 0.9431 - val_loss: 0.1573
Epoch 3/15
100/101 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9812 - loss: 0.0635

101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.9812 - loss: 0.0634 - val_accuracy: 0.9468 - val_loss: 0.1273
Epoch 4/15
100/101 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.9974 - loss: 0.0190

101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.9974 - loss: 0.0189 - val_accuracy: 0.9530 - val_loss: 0.1290
Epoch 5/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 39ms/step - accuracy: 1.0000 - loss: 0.0051 - val_accuracy: 0.9530 - val_loss: 0.1290
Epoch 6/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 0.0024

101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 1.0000 - loss: 0.0024 - val_accuracy: 0.9567 - val_loss: 0.1333
Epoch 7/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 0.9517 - val_loss: 0.1357
Epoch 8/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 1.0000 - loss: 8.8376e-04 - val_accuracy: 0.9530 - val_loss: 0.1433
Epoch 9/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 4s 38ms/step - accuracy: 1.0000 - loss: 6.2345e-04 - val_accuracy: 0.9542 - val_loss: 0.1509
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9529 - loss: 0.1510
Test accuracy: 0.9496


# Using Glove

In [21]:

glove_dir = '../'  # Change this to your GloVe path
embeddings_index = {}
with open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Create embedding matrix
embedding_matrix = np.zeros((max_features, embedding_dims))
for word, i in tokenizer.word_index.items():
    if i < max_features:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

## Training a model

In [22]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, Dropout, GlobalMaxPooling1D, Conv1D, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Embedding layer ( with glove!)
model = Sequential()
model.add(Input((maxlen,)))
model.add(Embedding(
    max_features, 
    embedding_dims,
    weights=[embedding_matrix],
    trainable=False  # Set to True to fine-tune embeddings
))


model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())

# Dense layers
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# Compile with better optimizer
model.compile(
    optimizer='adam',  # Adam typically works better than rmsprop
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


# Early stopping 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

# Save best model
model_checkpoint = ModelCheckpoint(
    'best_model_glove.h5',
    monitor='val_accuracy',
    save_best_only=True
)

# Train with callbacks
history = model.fit(
    x_train, y_train,
    epochs=15,  # More epochs, early stopping will prevent overfitting
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate on test set
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ (None, 500, 100)       │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 496, 128)       │        64,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_5          │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,072,449 (4.09 MB)

 Trainable params: 72,449 (283.00 KB)

 Non-trainable params: 1,000,000 (3.81 MB)

Epoch 1/15
100/101 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7268 - loss: 0.5098

101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.7290 - loss: 0.5067 - val_accuracy: 0.9134 - val_loss: 0.2071
Epoch 2/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.9333 - loss: 0.1637

101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.9334 - loss: 0.1636 - val_accuracy: 0.9282 - val_loss: 0.1792
Epoch 3/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9692 - loss: 0.0875

101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9692 - loss: 0.0875 - val_accuracy: 0.9307 - val_loss: 0.1782
Epoch 4/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9834 - loss: 0.0509 - val_accuracy: 0.9245 - val_loss: 0.1891
Epoch 5/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 27ms/step - accuracy: 0.9976 - loss: 0.0233 - val_accuracy: 0.9257 - val_loss: 0.1824
Epoch 6/15
101/101 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - accuracy: 0.9989 - loss: 0.0141 - val_accuracy: 0.9307 - val_loss: 0.1935
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9384 - loss: 0.1424
Test accuracy: 0.9397


# Predicting for the Competition

In [23]:
from tensorflow.keras import preprocessing

# Load the CSV with data to predict
prediction_csv_path = '../dataset1_inputs.csv'
df_predict = pd.read_csv(prediction_csv_path,sep="\t")

# Check the loaded data
print("Prediction dataset shape:", df_predict.shape)
print("Columns:", df_predict.columns)
print("Sample IDs:", df_predict['ID'].head())

# Preprocess the text data to match the training data format
# Note: Use the same tokenizer you used for training
# If you saved it with pickle, load it:
"""
import pickle
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
"""

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(df_predict['Text'].values)

# Pad sequences to the same length as during training
x_predict = preprocessing.sequence.pad_sequences(sequences, maxlen=maxlen)

# Make predictions with your trained model
predictions = model.predict(x_predict)

# Convert probabilities to binary labels (0 = Human, 1 = AI)
# Using 0.5 as the threshold - you might want to tune this based on your model's performance
labels = (predictions > 0.5).astype(int)

# Map labels to "AI" and "Human"
label_mapping = {1: "AI", 0: "Human"}
labels_mapped = [label_mapping[label] for label in labels.flatten()]

# Create a DataFrame with the results
results_df = pd.DataFrame({
    'ID': df_predict['ID'],
    'Label': labels_mapped
})

# Print sample of results
print("\nSample of prediction results:")
print(results_df.head())

# Save to CSV
output_csv_path = 'prediction_results.csv'
results_df.to_csv(output_csv_path,sep="\t",index=False)
print(f"\nResults saved to {output_csv_path}")

Prediction dataset shape: (30, 2)
Columns: Index(['ID', 'Text'], dtype='object')
Sample IDs: 0    D1-1
1    D1-2
2    D1-3
3    D1-4
4    D1-5
Name: ID, dtype: object
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

Sample of prediction results:
     ID  Label
0  D1-1  Human
1  D1-2  Human
2  D1-3     AI
3  D1-4     AI
4  D1-5  Human

Results saved to prediction_results.csv
